# Asteroid Classification

__Cem Sinan Ozturk__

_October 6, 2020_


In this project, I am going to perform a prediction analysis for classifying hazardous asteroids.

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display #to display all columns

import seaborn as sns
import matplotlib.pyplot as plt

import datetime
import time

from sklearn.metrics import accuracy_score, precision_recall_curve, plot_precision_recall_curve
from sklearn.metrics import roc_auc_score, precision_score, recall_score, average_precision_score
from sklearn.metrics import f1_score, plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import plot_roc_curve

from sklearn.feature_selection import SelectFromModel

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.dummy import DummyClassifier

# Models: 
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [ ]:
import warnings
#warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

## EDA (Exploratory Data Analysis):

Clean the data:
- Drop perfectly correlated columns(expect one of them)
- Columns with single values
- Close Approach Date vs Epoch Date Close Approach : Same information in a different representation. Drop one of them.

This data doesn't have `null` values but it is always better to make sure.



In [ ]:
df = pd.read_csv('./data/nasa.csv')

In [ ]:
df.shape

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

### Correlation between Features

As we can see from the following visualization, there are a number of columns which are perfectly correlated, and for most of the cases, these columns are the different representation of the same information. Therefore, it is best to eleminate extra ones.

For example, Est Diameter is given in km, m, miles and feet. We only need one of those.

In [ ]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

In [ ]:
col_list = list(df.columns)
extras = []
for col1 in df.columns:
    col_list.remove(col1)
    for col2 in col_list:
        if(df[col1].dtype in ["int64","float64"] and df[col2].dtype in ["int64","float64"]):
                if(abs(df[col1].corr(df[col2])) > 0.99):
                    print(col1,'-',col2, ':', df[col1].corr(df[col2]))
                    extras.append(col2)

In [ ]:
cols_to_delete = set(extras)
cols_to_delete

In [ ]:
df.drop(cols_to_delete, axis=1, inplace=True)

#### Columns with single value

Features with the same value for all the instances will not bring any value to our models. It is best to eleminate them.

In [ ]:
single_val_cols = [col for col in df.columns if df[col].nunique() == 1]
single_val_cols

In [ ]:
df.drop(single_val_cols, axis=1, inplace=True)

### Epoch Date Close Approach and Close Approach Date

`Epoch Date Close Approach` is epoch/unix timestamp representation of `Close Approach Date`. Therefore, we should only have one of those columns. For now, I will remove `Close Approach Date` since `Epoch Date Close Approach` can directly be used in the models.

Also, I may use the **year**, **month** and **day-of-year** values as features. These values may reveal any seasonal trend. (The same logic is relevant to other date columns) However, adding these features will require dummy variables (One-Hot-Encoding). It will greatly increase the number of variables.

In [ ]:
df.drop(['Epoch Date Close Approach'], axis=1, inplace=True) # or drop : Close Approach Date
#df.drop(['Close Approach Date'], axis=1, inplace=True) # or drop : Epoch Date Close Approach

In [ ]:
df['Close Approach Year'] = df['Close Approach Date'].astype('datetime64').dt.year.astype('str')
df['Close Approach Month'] = df['Close Approach Date'].astype('datetime64').dt.month.astype('str')
df['Close Approach Day of Year'] = df['Close Approach Date'].astype('datetime64').dt.dayofyear.astype('str')
df.drop(['Close Approach Date'], axis=1, inplace=True)

In [ ]:
#df['Close Approach Date'].astype('datetime64').hist()

In [ ]:
df['Close Approach Year'].astype('str')

### Orbit Determination Date:

Orbit Determination Date has values in the format of _"%Y-%m-%d %H:%M%S"_. Even if we are going to use this information for our model it seems using it in second precision is not what we may need. Since there are 2680 values in 4687 instance. Using in day precision may be a solution.

However, I will omit this date column for now since it seems it is just the information of when the date is put onto the system. 

I may need to revisit here to explore this column, later.

In [ ]:
df['Orbit Determination Year Month'] = df['Orbit Determination Date'].astype('datetime64').dt.strftime('%Y-%m')
df['Orbit Determination Year Month'].hist(xrot=40)
plt.ylabel('Number of Instances')
plt.xlabel('Date(Month-Year)')
plt.title('Orbit Determination Year Month')

In [ ]:
df.drop(['Orbit Determination Date','Orbit Determination Year Month'], axis=1, inplace=True)

### Pairplot for Remaining Features

It is a crowded plot with too many features but it may be helpful for a quick look. We can see there are some candidates for outliers but I will keep it simple and skip this for now. 

Next plots are memory and time intensive so I will keep them as comments. (I ran it only once, normally I would not keep it in my analysis notebook)

In [ ]:
#sns.pairplot(df, kind='reg', diag_kind='kde') 

Take a look on the large diameter (`Est Dia in KM(min)>3`) instances to see if there is anything to catch with eye.

In [ ]:
df[df['Est Dia in KM(min)']>3]

For example, there one instance with `Est Dia in KM(min) >15` which is way out of the range of `Est Dia in KM(min)`. It has mean 0.2 and standard deviation of 0.35. Although there are other candidates it seems a very obvious outlier from the plots below. I will keep it for modelling but I will create a copy for visualization purposes in the below.

In [ ]:
#dfcopy = df[df['Est Dia in KM(min)']<15]
#sns.pairplot(dfcopy, kind='reg', diag_kind='kde') 

### Preprocessing the columns

There are some preprocessing steps. In normal circumstances, missing values needs to be imputed. Here, no need for this step.

Also, scaling or encoding would be other types of preprocessing steps that I will perform here. 

In [ ]:
y = df.loc[:, df.columns == 'Hazardous'].astype(int)
X = df.loc[:, df.columns != 'Hazardous']

In [ ]:
numeric_features = X.select_dtypes(exclude="object").columns
categorical_features =  X.select_dtypes(include="object").columns

In [ ]:
numeric_transformer = Pipeline(steps=[
                                      ('scaler', StandardScaler())
                                    ])


categorical_transformer = Pipeline(steps=[
                                          ('onehot', OneHotEncoder(handle_unknown='ignore'))
                                         ])

preprocessor = ColumnTransformer(
                                 transformers=[
                                    ('num', numeric_transformer, numeric_features),
                                    ('cat', categorical_transformer, categorical_features)
                                ])

### Model Selection and Fitting:



In [ ]:
hazardous_perc = np.mean(df['Hazardous'])*100
print("%0.2f" %hazardous_perc, "% of the total astroids are hazardous in the data.")

As we can see, only 16.1% of the asteroids are hazardous. Therefore, the data is unbalanced, and we need to be careful when we are evaluating the models. For example, if a model predicts everything as non-hazardous and missed all the hazardous asteroids. The accuracy will be 83.9% but it is equal to doing nothing! Therefore, we need to keep this in mind.

### Split the Data into Training, Test, and Validation Sets

In [ ]:
X_trainvalid, X_test, y_trainvalid, y_test = train_test_split(X, y, train_size=0.8, random_state=13)
X_train, X_valid, y_train, y_valid = train_test_split(X_trainvalid, y_trainvalid, 
                                                          train_size=0.75, random_state=13)

### Accuracy Measures

In this example, we have unbalanced data, we need to be careful on accuracy. Since, number of correct predicted labels may not mean model is predicting useful results. Therefore, we have to be careful and get use of other measures from confusion matrix such as precision, recall or f1 score.  Since we need to be able to predict positive class (**Hazardous**) in this unbalanced data. Here the formulas for better remembering. 

$$Recall =  \frac{TP}{TP+FN}$$

$$Precision =  \frac{TP}{TP+FP}$$

$$F_1 =  \frac{2* Precision * Recall}{Precision * Recall} $$

In [ ]:
def get_scores(model, 
                X_train, y_train,
                X_valid, y_valid, 
                show = True
               ):
    """
    Returns train and validation error given a model
    train and validation X and y portions
    Parameters
    ----------
    model: sklearn classifier model
        The sklearn model
    X_train: numpy.ndarray        
        The X part of the train set
    y_train: numpy.ndarray
        The y part of the train set    
    X_valid: numpy.ndarray        
        The X part of the validation set
    y_valid: numpy.ndarray
        The y part of the validation set    
    Returns
    -------
        train_err: float
        test_err: float            
    """ 
    #y_pred = model.predict(X_valid)
    #auc = roc_auc_score(y_valid, y_pred)
    train_err = (1-model.score(X_train, y_train))
    valid_err = (1-model.score(X_valid, y_valid))
    
    if show: 
        print("Training error:   %.2f" % train_err)
        print("Validation error: %.2f" % valid_err)
     #   print("ROC-Area Under Curve: %.2f" % auc )
        print('\n')
    return train_err, valid_err  # ,auc

In [ ]:
def display_confusion_matrix_classification_report(model, X_valid, y_valid, 
                                                   labels=['Non hazardous', 'Hazardous']):
    """
    Displays confusion matrix and classification report. 
    
    Arguments
    ---------     
    model -- sklearn classifier model
        The sklearn model
    X_valid -- numpy.ndarray        
        The X part of the validation set
    y_valid -- numpy.ndarray
        The y part of the validation set       

    Keyword arguments:
    -----------
    labels -- list (default = ['Non fraud', 'Fraud'])
        The labels shown in the confusion matrix
    Returns
    -------
        None
    """
    ### Display confusion matrix 
    disp = plot_confusion_matrix(model, X_valid, y_valid,
                                 display_labels=labels,
                                 cmap=plt.cm.Blues, 
                                 values_format = 'd')
    disp.ax_.set_title('Confusion matrix for the dataset')

    ### Print classification report
    print(classification_report(y_valid, model.predict(X_valid)))

#### Dummy Model for Baseline

It will be useful to compare real classifiers. 

For more info, [sklearn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyClassifier.html).

In [ ]:
# Lets create an empty dictionary to store all the results
results_dict = {}

In [ ]:
numeric_transformer = Pipeline(steps=[
                                    # ('imputer', SimpleImputer(strategy='median')),
                                      ('scaler', StandardScaler())
                                    ])


categorical_transformer = Pipeline(steps=[
                                          #('imputer', SimpleImputer(strategy='constant', 
                                          #                          fill_value='missing')),
                                          ('onehot', OneHotEncoder(handle_unknown='ignore'))
                                         ])

preprocessor = ColumnTransformer(
                                 transformers=[
                                    ('num', numeric_transformer, numeric_features),
                                    ('cat', categorical_transformer, categorical_features)
                                ])

In [ ]:
print('Fitting baseline model: ')
dummy = DummyClassifier()

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', dummy)])

t = time.time()
clf.fit(X_train, y_train)
t_fit = time.time() 
tr_err, valid_err = get_scores(clf, X_train, y_train, X_valid, y_valid)
cross_val = cross_val_score(clf, X_train, y_train, cv=5)
y_pred = clf.predict(X_valid)
#auc= roc_auc_score(y_valid, y_pred)
f1_sc = f1_score(y_valid, y_pred)
t_predict = time.time() 

time_to_fit = t_fit - t
time_to_predict = t_predict - t_fit
    

results_dict['dummy'] = [round(tr_err,3), "%0.2f (+/- %0.2f)" %(cross_val.mean(), cross_val.std() * 2), round(valid_err,3), round(f1_sc,3), round(time_to_fit,4), round(time_to_predict,4)]

## Classifiers to Evaluate:

In [ ]:
models = {
          'decision tree': DecisionTreeClassifier(),
          'kNN': KNeighborsClassifier(),
          'logistic regression': LogisticRegression(),
          'RBF SVM' : SVC(), 
          'random forest' : RandomForestClassifier(), 
          'xgboost' : XGBClassifier(),
          'lgbm': LGBMClassifier()
         }

for model_name, model in models.items():
    t = time.time()
    #print(model_name, ":")    
    clf = Pipeline(steps=[('preprocessor', preprocessor),
                          ('classifier', model)])

    clf.fit(X_train, y_train)
    tr_err, valid_err = get_scores(clf, X_train, y_train, 
                                   X_valid, y_valid, show = False)
    cross_val = cross_val_score(clf, X_train, y_train, cv=5)
    t_fit = time.time() 
    y_pred = clf.predict(X_valid)
    #auc= roc_auc_score(y_valid, y_pred)
    f1_sc = f1_score(y_valid, y_pred)
    t_predict = time.time() 
    
    time_to_fit = t_fit - t
    time_to_predict = t_predict - t_fit
    
    results_dict[model_name] = [round(tr_err,3),"%0.2f (+/- %0.2f)" % (cross_val.mean(), cross_val.std() * 2), round(valid_err,3), round(f1_sc,3), round(time_to_fit,4), round(time_to_predict,4)]
    #print("Elapsed time: %.1f s" % elapsed_time)

In [ ]:
results_df = pd.DataFrame(results_dict).T
results_df.columns = ["Train error","Training CV Accuracy",  "Validation error", "F1 Score on Validation", "Fit Time in sec", "Predict Time in sec"]
results_df

- SVM and kNN are slow models. So, it is best to eleminate them. Random Forest is also slow to fit and it is overfitted. 
- Decision Tree, Random Forest and boosted models(xgboost and lgbm) seems overfitted. Training errors are 0. We can apply hyperparameter optimization.

I prefer to continue with Logistic Regression. It is the fastest model with reasonable accuracy and AUC values.


### Hyperparameter optimization on Selected Model (Logistic Regression)

In [ ]:
# Hyperparameter optimization 

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression())])

### We can also choose optimal class_weight ratio using grid search. 
weights = np.linspace(0.05, 0.95, 20)

param_grid = {
    'classifier__C': [0.01, 0.1, 1.0, 10, 100],
    #'classifier__penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'classifier__class_weight': [{0: x, 1: 1.0-x} for x in weights]
}

grid_search = GridSearchCV(clf,
                           param_grid=param_grid,
                           scoring='f1',
                           cv=10)
grid_search.fit(X_trainvalid, y_trainvalid)

print(("best logistic regression from grid search (F1 Score): %.3f"
       % grid_search.score(X_valid, y_valid)))

In [ ]:
print("Best parameters : %s" % grid_search.best_params_)

In [ ]:
display_confusion_matrix_classification_report(grid_search, X_valid, y_valid, 
                                                   labels=['Non hazardous', 'Hazardous'])

Validation scores of 0.89 precision and 0.88 recall on hazardous class seems reasonably well. I will continue to use this Logistic Regression model for predicting unseen test data.

### Final Results on the test data:

In [ ]:
display_confusion_matrix_classification_report(grid_search, X_test, y_test, 
                                                   labels=['Non hazardous', 'Hazardous'])

Both precision and recall for the hazardous class have fallen to 0.83 and 0.82, respectively. Also, the accuracy score has fallen from 96% to 94%. However, the model results still seem reasonable. Based on our test set, we will be able to identify 132 (~83%) hazardous asteroids out of a total of 159 (132+27) hazardous asteroids. I will come with the cost of 29 *non-hazardous* to be misidentified as *hazardous*. 

If we want to increase the precision of classifying hazardous asteroids, it will cost a reduction in recall which means we will have more *non-hazardous* to be misidentified as *hazardous*. In the case of NASA, if we have more asteroids predicted as hazardous, it may increase the cost of tracking those asteroids and other studies. However, it will give peace of mind since a missed hazardous asteroid may result in massive natural destruction.

## Improvements and/or Other Approaches

Since we have unbalanced data, we could also perform:

- Undersampling: It is mainly undersampling the "non-hazardous" class to get balanced data and fit model on this.
- Oversampling: It is oversampling the minority class("hazardous"). There are different techniques for oversampling. One is random oversampling weith replacement. Another model is SMOTE (Synthetic Minority Over-sampling Technique). SMOTE creates "synthetic" instances instead of oversampling with replacement.

Here, I have selected Logistic Regression since it is fast and interpretable model. Boosted models with hyperparameter optimization will tends to give better results. This type of models can be another way to go. 

Last but not least, making the analysis reproducible by automating some processes would be beneficial for future similar works. For the sake of simplicity, I keep all the code and analysis in a single JupyterNotebook. For the best practices, my next step would be breakdown down the code into scripts.